In [1]:
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
raw_data=pd.read_json("C:/Users/dharm/Desktop/Data Engineering]/receipts.json/receipts.json",lines=True)

# Understanding Raw data

In [3]:
row_count=raw_data.shape[0]
col_count=raw_data.shape[1]
print("Row count:",row_count,"||", "col count:",col_count)

Row count: 1119 || col count: 15


In [4]:
raw_data.head(5)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [5]:
raw_data=raw_data.explode('rewardsReceiptItemList')
raw_data.reset_index(inplace=True)


In [8]:
from ast import literal_eval
raw_data=raw_data.fillna({'rewardsReceiptItemList':'{}'})
raw_data['rewardsReceiptItemList']=raw_data['rewardsReceiptItemList'].apply(lambda x:str(x))
raw_data['rewardsReceiptItemList']=raw_data['rewardsReceiptItemList'].apply(literal_eval)

In [9]:
receipts_norm=pd.json_normalize(raw_data['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList').add_prefix('rewardsReceiptItemList')

In [10]:
receipts_final=pd.merge(raw_data,receipts_norm,left_index=True,right_index=True,how='outer')

In [11]:
receipts_final

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemListitemNumber,rewardsReceiptItemListoriginalMetaBriteQuantityPurchased,rewardsReceiptItemListpointsEarned,rewardsReceiptItemListtargetPrice,rewardsReceiptItemListcompetitiveProduct,rewardsReceiptItemListoriginalFinalPrice,rewardsReceiptItemListoriginalMetaBriteItemPrice,rewardsReceiptItemListdeleted,rewardsReceiptItemListpriceAfterCoupon,rewardsReceiptItemListmetabriteCampaignId
0,0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


# Formating rows that has dictionaries

In [14]:
set().union(*(d.keys() for d in receipts_final['_id']))

{'$oid'}

In [15]:
set().union(*(d.keys() for d in receipts_final['createDate']))

{'$date'}

In [16]:
set().union(*(d.keys() for d in receipts_final['dateScanned']))

{'$date'}

In [19]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['finishedDate'])['finishedDate']))

{'$date'}

In [20]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['modifyDate'])['modifyDate']))

{'$date'}

In [21]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['pointsAwardedDate'])['pointsAwardedDate']))

{'$date'}

In [22]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['purchaseDate'])['purchaseDate']))

{'$date'}

# covert UTC timestamp

In [23]:
from datetime import datetime

def date_converter(x):
    try:
        return datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S')
    except (TypeError, ValueError):
        return None

receipts_final['_id'] = receipts_final['_id'].apply(lambda x: x['$oid'])
date_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for col in date_columns:
    receipts_final[col] = receipts_final[col].apply(date_converter)


In [24]:
receipts_final.head()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemListitemNumber,rewardsReceiptItemListoriginalMetaBriteQuantityPurchased,rewardsReceiptItemListpointsEarned,rewardsReceiptItemListtargetPrice,rewardsReceiptItemListcompetitiveProduct,rewardsReceiptItemListoriginalFinalPrice,rewardsReceiptItemListoriginalMetaBriteItemPrice,rewardsReceiptItemListdeleted,rewardsReceiptItemListpriceAfterCoupon,rewardsReceiptItemListmetabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",None,None,None,None,None,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",None,None,None,None,None,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",None,None,None,None,None,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,None,None,None,None,None,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,None,None,None,None,None,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
receipts_final['rewardsReceiptItemListpointsPayerId'].isnull().sum()

6114